In [1]:
import os
import ray
import time
import math
import numpy as np
import pandas as pd
from ray import tune
import seaborn as sns
from typing import Any
import gymnasium as gym
from copy import deepcopy
import plotly.express as px
from gymnasium import spaces
from pettingzoo import AECEnv
import matplotlib.pyplot as plt
from ray.rllib.env import PettingZooEnv
from ray.tune.logger import pretty_print
from ray.rllib.algorithms.ppo import PPOConfig
# from models.PyFlytModel_MOG import PyFlytModel_MOG
# from models.PyFlytModel_ENN import PyFlytModel_ENN
from ray.rllib.utils.framework import try_import_torch
from policies.ppo_torch_policy import SimpleTorchPolicy
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from models.SimpleTorchModel import SimpleCustomTorchModel
from add_ons.normalize_advantages import NormalizeAdvantagesCallback


import PyFlyt.gym_envs
from ray.tune.registry import register_env
from PyFlyt.gym_envs import FlattenWaypointEnv
from PyFlyt.gym_envs.quadx_envs import quadx_hover_env, quadx_waypoints_env
from PyFlyt.pz_envs.fixedwing_envs.ma_fixedwing_dogfight_env import MAFixedwingDogfightEnv

pybullet build time: Nov 28 2023 23:45:17


In [2]:
path = os.getcwd()
torch, nn = try_import_torch()
ray.init()

2024-07-03 15:46:40,888	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.34gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-07-03 15:46:42,036	INFO worker.py:1771 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.31.0


(RolloutWorker pid=2674) pybullet build time: Nov 28 2023 23:45:17


(RolloutWorker pid=2676)                              
(RolloutWorker pid=2676)                               [repeated 37x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(RolloutWorker pid=2678)                               [repeated 145x across cluster]
(RolloutWorker pid=2676)                               [repeated 93x across cluster]
(RolloutWorker pid=2682)                               [repeated 84x across cluster]
(RolloutWorker pid=2680)                               [repeated 90x across cluster]
(RolloutWorker pid=2677)                               [repeated 92x across cluster]
(RolloutWorker pid=2673)                               [repeated 55x across cluster]
(RolloutWorker pid=2677)                               [repeated 52x across cluster]
(RolloutWorker pid=2678)                           

In [3]:
class CustomDogfightEnv(MultiAgentEnv):
    def __init__(self, 
                 config, 
                 env: AECEnv = None):

        super().__init__()
        if env is None:
            self.env = MAFixedwingDogfightEnv()
        else:
            self.env = env
        self.env.reset()
        
        self.agent_ids = self.env.possible_agents
        self.observation_space = self.env.observation_space(self.env.agents[0])
        self.action_space = self.env.action_space(self.env.agents[0])
        self.results = []

        assert all(
            self.env.observation_space(agent) == self.observation_space
            for agent in self.env.agents
        ), (
            "Observation spaces for all agents must be identical. Perhaps "
            "SuperSuit's pad_observations wrapper can help (useage: "
            "`supersuit.aec_wrappers.pad_observations(env)`"
        )

        assert all(
            self.env.action_space(agent) == self.action_space
            for agent in self.env.agents
        ), (
            "Action spaces for all agents must be identical. Perhaps "
            "SuperSuit's pad_action_space wrapper can help (usage: "
            "`supersuit.aec_wrappers.pad_action_space(env)`"
        )
        self._agent_ids = set(self.env.agents)


    def reset(self, seed=None, options=None):
        observations, infos = self.env.reset()
        
        # Ensure observations match the expected shape
        formatted_observations = {
            agent_id: self._format_observation(obs)
            for agent_id, obs in observations.items()
        }
        
        return formatted_observations, infos

    def step(self, action_dict):
        observations, rewards, terminations, truncations, infos = self.env.step(action_dict)
        
        # Ensure observations match the expected shape
        formatted_observations = {
            agent_id: self._format_observation(obs)
            for agent_id, obs in observations.items()
        }

        # Ensure "__all__" keys are present in terminations and truncations dictionaries
        terminations["__all__"] = any(terminations.values())
        truncations["__all__"] = any(truncations.values())

        for agent_id, reward in rewards.items():
            # print(f"Agent {agent_id} reward: {reward}")
            self.results.append([agent_id, reward])

        return formatted_observations, rewards, terminations, truncations, infos

    # def step(self, action_dict):
    #     observations, rewards, terminations, truncations, infos = self.env_step(action_dict)
        
    #     # Ensure observations match the expected shape
    #     formatted_observations = {
    #         agent_id: self._format_observation(obs)
    #         for agent_id, obs in observations.items()
    #     }
    
    #     # Ensure "__all__" keys are present in terminations and truncations dictionaries
    #     terminations["__all__"] = any(terminations.values())
    #     truncations["__all__"] = any(truncations.values())
    
    #     for agent_id, reward in rewards.items():
    #         # Print or log rewards if needed
    #         self.results.append([agent_id, reward])
    
    #     return formatted_observations, rewards, terminations, truncations, infos
    
    # def env_step(self, actions: dict[str, np.ndarray]) -> tuple[
    #         dict[str, Any], dict[str, float], dict[str, bool], dict[str, bool], dict[str, dict[str, Any]]
    #     ]:
    #     """Custom step method for the environment."""
    #     self.env.past_actions = deepcopy(self.env.current_actions)
    #     self.env.current_actions *= 0.0
    #     for k, v in actions.items():
    #         self.env.current_actions[self.env.agent_name_mapping[k]] = v
    #     self.env.aviary.set_all_setpoints(self.env.current_actions)
    
    #     observations = dict()
    #     terminations = {k: False for k in self.env.agents}
    #     truncations = {k: False for k in self.env.agents}
    #     rewards = {k: 0.0 for k in self.env.agents}
    #     infos = {k: dict() for k in self.env.agents}
    
    #     for _ in range(self.env.env_step_ratio):
    #         self.env.aviary.step()
    #         for ag in self.env.agents:
    #             ag_id = self.env.agent_name_mapping[ag]
    #             term, trunc, rew, info = self.env.compute_term_trunc_reward_info_by_id(ag_id)
    #             terminations[ag] |= term
    #             truncations[ag] |= trunc
    #             rewards[ag] += rew
    #             infos[ag] = {**infos[ag], **info}
    #             observations[ag] = self.env.compute_observation_by_id(ag_id)
    
    #     self.env.step_count += 1
    #     self.env.agents = [
    #         agent for agent in self.env.agents
    #         if not (terminations[agent] or truncations[agent])
    #     ]
    
    #     return observations, rewards, terminations, truncations, infos


    def _format_observation(self, observation):
        # Ensure the observation has the correct shape (34,)
        if observation.shape[0] == 30:
            # Add placeholders for missing components to match the expected shape
            observation = np.concatenate((observation, np.zeros(4)))
        return observation
        
    def get_results(self):
        return self.results


def env_creator(config):
    return CustomDogfightEnv(config)
register_env('MAFixedwingDogfightEnv', env_creator)


In [4]:
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    # Check if agent_id is a digit
    if agent_id.isdigit():
        return 'policy_1' if int(agent_id) % 2 == 0 else 'policy_2'
    # Handle agent_ids like 'uav_0', 'uav_1', etc.
    return 'policy_1' if int(agent_id.split('_')[1]) % 2 == 0 else 'policy_2'

In [5]:
env_config = {
    'spawn_height': 5.0,
    'damage_per_hit': 0.02,
    'lethal_distance': 15.0,
    'lethal_angle_radians': 0.1,
    'assisted_flight': True,
    'sparse_reward': False,
    'flight_dome_size': 150.0,
    'max_duration_seconds': 60.0,
    'agent_hz': 30,
    'render_mode': None,
}

In [ ]:
%%time

env_example = env_creator(env_config)
obs_space = env_example.observation_space
action_space = env_example.action_space

config = PPOConfig().training(
    gamma = 0.99,
    lambda_ = 0.95,
    # kl_coeff = 0.5,
    num_sgd_iter = 15,
    # lr_schedule = [[0, 0.0003], [15_000_000, 0.00025], [30_000_000, 0.0002], [50_000_000, 0.0001]],
    lr = 0.0001,
    vf_loss_coeff = 0.5,
    vf_clip_param = 15.0,
    clip_param = 0.2,
    grad_clip_by ='norm', 
    train_batch_size = 12_000, 
    sgd_minibatch_size = 3_000,
    grad_clip = 0.5,
    # model = {'custom_model': 'SimpleCustomTorchModel', 
    #        'vf_share_layers': False,
    #        'fcnet_hiddens': [256,256],
    #        'fcnet_activation': 'LeakyReLU',
    #          #this isn't used for some models, but doesn't hurt to keep it
    #        'custom_model_config': {
    #             'num_gaussians': 2,
    #            'num_outputs': action_space.shape[0]
    #        }
    #         }
).environment(
    env = 'MAFixedwingDogfightEnv',
    env_config = env_config
).rollouts(
num_rollout_workers = 10
).resources(num_gpus = 1
)

# .callbacks(NormalizeAdvantagesCallback
# )

# .multi_agent(
#     policies = {
#         'policy_1': (SimpleTorchPolicy, obs_space, action_space, {}),
#         'policy_2': (SimpleTorchPolicy, obs_space, action_space, {}),
#     },
#     policy_mapping_fn=policy_mapping_fn
# )

# analysis = tune.run(
#     'PPO',
#     config=config.to_dict(),
#     stop={'training_iteration':300},
#     checkpoint_freq=10,
#     checkpoint_at_end=True,
#     # local_dir='./ray_results'
# )


algo = config.build()

num_iterations = 750
results = []

for i in range(num_iterations):
    result = algo.train()
    if i % 10 == 0:
        print(f"Iteration: {i}, Mean Reward: {result['env_runners']['episode_reward_mean']} episode length: {result['env_runners']['episode_len_mean']}")
    results.append([result['env_runners']['episode_reward_mean'], result['env_runners']['episode_len_mean']])

results_df = pd.DataFrame(results)

ray.shutdown()


2024-07-03 15:46:45,228	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-07-03 15:46:45,229	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:516: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by set

Iteration: 0, Mean Reward: -6803.635425456339 episode length: 69.85365853658537
Iteration: 10, Mean Reward: -7103.871958255148 episode length: 209.57
Iteration: 20, Mean Reward: -6633.1796678556575 episode length: 311.06
Iteration: 30, Mean Reward: -6448.8785194567345 episode length: 392.73
Iteration: 40, Mean Reward: -5326.017341203835 episode length: 540.67
Iteration: 50, Mean Reward: -3673.7150671760423 episode length: 656.91
Iteration: 60, Mean Reward: -4696.046862562452 episode length: 581.71
Iteration: 70, Mean Reward: -4980.117233410088 episode length: 577.38
Iteration: 80, Mean Reward: -5221.057219914584 episode length: 441.16
Iteration: 90, Mean Reward: -5542.366063524643 episode length: 480.72
Iteration: 100, Mean Reward: -5784.43288166485 episode length: 427.91
Iteration: 110, Mean Reward: -5824.259987194375 episode length: 369.04
Iteration: 120, Mean Reward: -5626.098049820695 episode length: 394.63
Iteration: 130, Mean Reward: -5686.113734697257 episode length: 413.87
Iter

In [ ]:
results_df = pd.DataFrame(results)
experiment_type = 'enn_2dim'
results_df.to_csv(path + '/logs/test_runs/'+experiment_type+'.csv')

In [ ]:
results

In [ ]:
plt.plot(results)
plt.title('Training Progress - Mean Reward per Episode')
plt.xlabel('Iteration')
plt.ylabel('Mean Reward')
# plt.savefig('Basic PPO - HalfCheetah-v4')
plt.show()

In [ ]:
algo.logdir

In [ ]:
env = FlattenWaypointEnv(gym.make(id='PyFlyt/QuadX-Waypoints-v1', flight_mode=-1), context_length=1)

obs_list = []
obs, info = env.reset()
# env.env.env.env.env.drones[0].set_mode(-1)
targets = env.unwrapped.waypoints.targets
points = np.concatenate((obs[10:13].reshape(-1,3), targets))
obs = {'default': obs}
obs_list += [obs]

reward_list = []
action_list = []
start = time.time()
for i in range(10*40):
    compute_action = algo.compute_actions(obs)
    action = compute_action['default']
    # obs, reward, terminated, truncated, info = env.step(np.zeros((4))+.79)
    obs, reward, terminated, truncated, info = env.step(action)

    obs = {'default': obs}
    
    obs_list += [obs]
    
    reward_list += [reward]
    action_list += [action]
    
    if terminated or info['num_targets_reached'] == 4:
        break

arrays = [d['default'] for d in obs_list]
obs_array = np.vstack(arrays)
reward_array = np.array(reward_list)
action_array = np.array(action_list) 
env.close()

In [ ]:
plotly_figure = px.scatter_3d(x=obs_array[:,10], y=obs_array[:,11], z=obs_array[:,12], opacity=.6, color=np.arange(len(obs_array)))
plotly_figure.add_scatter3d(x=targets[:,0], y=targets[:,1], z=targets[:,2], marker={'color':'green', 'symbol':'square-open', 'size':25, 'line':{'width':10}}, mode='markers')
plotly_figure.write_html(path+'/3D_renders/3d_drone_space4_'+experiment_type+'.html')

In [ ]:
import seaborn as sns

In [ ]:
dataframes = {}
for filename in os.listdir(path+'/logs/test_runs'):
    if filename.endswith('.csv'):
        file_path = os.path.join(path+'/logs/test_runs', filename)
        df = pd.read_csv(file_path)
        key = os.path.splitext(filename)[0]
        dataframes[key] = df


data_list = []
labels = []
output_desired = 'length' #else will give length

for key, df in dataframes.items():
    if output_desired == 'reward':
        data_list.append(df.iloc[:,0])
        labels.append(f"reward for {key}")
    else:
        data_list.append(df.iloc[:,1])
        labels.append(f"length for {key}")

for data in data_list:
    sns.kdeplot(data, fill = True)

plt.legend(title = 'Modes', labels = labels)
plt.title(f"{output_desired}")
plt.show()

In [ ]:
dataframes = {}
for filename in os.listdir(path+'/logs/test_runs'):
    if filename.endswith('.csv'):
        file_path = os.path.join(path+'/logs/test_runs', filename)
        df = pd.read_csv(file_path)
        key = os.path.splitext(filename)[0]
        dataframes[key] = df


reward = []
labels = []
output_desired = 'reward' #else will give length

for key, df in dataframes.items():
    plt.scatter(df.iloc[:,0], df.iloc[:,1])
    labels.append(f"length for {key}")

plt.legend(title = 'Different runs', labels = labels)
plt.title(f"{output_desired} over time")
plt.show()